In [2]:
import sys
import os

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.preprocessing.extract import recursive_extract_text

recursive_extract_text('articles/raw/')

Processing articles/raw/Building a Massively Scalable Serverless Chat Application with AWS AppSync _ by Sarah Hamilton _ Serverless Transformation _ Medium.pdf
Processing articles/raw/Serverless webhooks — Designing Lift _ by Frédéric Barthelet _ Serverless Transformation _ Medium.pdf
Processing articles/raw/Building a Serverless Anti-Corruption Layer with CDK _ by Gregory Torrington _ Serverless Transformation _ Medium.pdf
Processing articles/raw/Building Serverless Observability Tools With Custom Metrics and Dashboards in CDK _ by Luke Yianni _ Serverless Transformation _ Medium.pdf
Processing articles/raw/Why Serverless will enable the Edge Computing Revolution _ by Ben Ellerby _ Serverless Transformation _ Medium.pdf
Processing articles/raw/Serverless Image Object Detection at a Social Media Startup _ by Aiden Walton _ Serverless Transformation _ Medium.pdf
Processing articles/raw/Enabling the Optimal Serverless Platform Team — CDK and Team Topologies _ by Ben Ellerby _ Serverless 

In [3]:
from src.preprocessing.extract import extract_text

pdf_path = 'articles/raw/Building a Massively Scalable Serverless Chat Application with AWS AppSync _ by Sarah Hamilton _ Serverless Transformation _ Medium.pdf'
output_path = 'Building a Massively Scalable Serverless Chat Application with AWS AppSync _ by Sarah Hamilton _ Serverless Transformation _ Medium.txt'

extract_text(pdf_path, output_path)


In [4]:
from src.preprocessing.chunk import recursive_chunk_text
from src.preprocessing.chunk import chunk_text
import os

# delete articles/chunks/chunks.txt
if os.path.exists('articles/chunks/chunks.txt'):
    os.remove('articles/chunks/chunks.txt')
    
# recursive_chunk_text('articles/parsed/')
chunk_text('articles/parsed/Building a Massively Scalable Serverless Chat Application with AWS AppSync _ by Sarah Hamilton _ Serverless Transformation _ Medium.txt')

In [1]:
from src.preprocessing.neutralising.llm import NeutraliserLLM

neutraliser = NeutraliserLLM('articles/chunks/chunks.txt')
neutralised_chunks = neutraliser.neutralise_chunks()

Neutralising chunk: 1/7
Neutralising chunk: 2/7
Neutralising chunk: 3/7
Neutralising chunk: 4/7
Neutralising chunk: 5/7
Neutralising chunk: 6/7
Neutralising chunk: 7/7


In [4]:
with open('articles/neutralised/neutralised_chunks.txt', 'w') as f:
    for chunk in neutralised_chunks:
        f.write(chunk + '\n')
        